In [249]:
from pymongo import MongoClient
import pandas as pd
import pprint
import json
import pandas as pd
import requests
from pandas import DataFrame
import numpy as np
#from cleaning.ipynb import*

In [250]:
client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [251]:
#cleaned_companies
companies = pd.DataFrame(list(db.companies_coor.find({},{"products":0, "twitter_username":0,"index":0,
                                                      "level_0":0,"_id":0, "founded_day":0, "founded_month":0})))

#cleaned_neighbors (for the moment working with NY neighbors)
neighbors = pd.DataFrame(list(db.neighborhood.find({})))

In [252]:
companies.shape

(9895, 8)

In [253]:
neighbors.head(3)

,_id,geometry,name
0,55cb9c666c522cafdb053a1a,"{'coordinates': [[[-73.94193078816193, 40.7007...",Bedford
1,55cb9c666c522cafdb053a1b,"{'coordinates': [[[-73.94732672160579, 40.6291...",Midwood
2,55cb9c666c522cafdb053a1c,"{'coordinates': [[[-73.89138023380261, 40.8617...",Fordham South


def findNear(geopoint, radio_max_meters=200):
    return db.companies_coor.find({
        "location": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })
geopoint = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]  #NY, 133 offices
}
city = "city"
radio_max_meters = 1000
num_offices = findNear(geopoint, 1000).count()
print(f"Hay {num_offices} oficinas cerca de {city} a {radio_max_meters} metros")

In [254]:

# How to biuld a matrix with all the companies > search per neighbor

# pasarle una lista de registros y que me reciba un valor con el número de ellas que tiene cerca
# aquí busco los máximos

            #Ranking weighted averages in MongoDB
# ponderar con más valor las que se dedican a :
        # categories: games_video,web,social,ecommerce,software
        # < de 10 años
    

In [255]:
#Ranking weighted averages   #1   category_code
def pondera_v(cat): 
    suma = 0
    if cat == "games_video":
        suma = 4
    if cat == "web":
        suma = 3
    if cat == "music":
        suma == 2
    return suma
companies ['category_p'] = companies["category_code"].apply(pondera_v)

In [270]:
#Ranking weighted averages   #2   closest companies

def findNear(geopoint, radio_max_meters=200):
    return db.companies_coor.find({
        "location": {
         "$near": {
           "$geometry": geopoint,
           "$maxDistance": radio_max_meters,
         }
       }
    })

def findNear_category(df, radio_max_meters=1000):
    #filter by city if desired
    #city = input(" Enter city: ")
    #df_filtered = df[df['city']==city]
    
    lstng = []
    for row in range(len(df)):
        geopoint = df.location.iloc[row]
        
        #sort by closest companies from loc  
        lst = len(list(findNear(geopoint,)))
        lstng.append(lst)
        df_overSF=pd.DataFrame(lstng)
    return lstng, df_overSF

lstng_a, df_overSF = (findNear_category(companies))
# Group by “source”, and count the total number of “source”

In [268]:
companies.join(df_overSF)

,category_code,city,currency_n,founded_year,latitud,location,longitud,name,category_p,0
0,web,San Francisco,1.85e+07,2002.0,37.775196,"{'type': 'Point', 'coordinates': [-122.419204,...",-122.419204,StumbleUpon,3,59
1,web,West Hollywood,1.65e+07,2006.0,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",-118.393064,Geni,3,1
2,games_video,Culver City,0,NaN,34.025958,"{'type': 'Point', 'coordinates': [-118.379768,...",-118.379768,Flektor,4,3
3,music,San Diego,7.31e+07,2006.0,33.022176,"{'type': 'Point', 'coordinates': [-117.081406,...",-117.081406,Slacker,0,1
4,news,San Francisco,2.58e+07,2007.0,37.789634,"{'type': 'Point', 'coordinates': [-122.404052,...",-122.404052,Scribd,0,19
5,social,New York City,5.58e+06,2007.0,40.757929,"{'type': 'Point', 'coordinates': [-73.985506, ...",-73.985506,MeetMoi,0,39
6,games_video,New York,4.5e+07,2006.0,40.746497,"{'type': 'Point', 'coordinates': [-74.0094471,...",-74.009447,Joost,4,4
7,web,San Carlos,0,1999.0,37.506885,"{'type': 'Point', 'coordinates': [-122.247573,...",-122.247573,Postini,3,3
8,news,San Francisco,4.5e+07,2004.0,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",-122.394523,Digg,0,1
9,web,New York,0,1971.0,40.757725,"{'type': 'Point', 'coordinates': [-73.986011, ...",-73.986011,Viacom,3,41


In [281]:
#Ranking weighted averages   #3   founded year
def find_founded_year(df, radio_max_meters=2000):
    lstng = []
    for row in range(len(df)): 
        query=db.companies_coor.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df[row],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'founded_year':{'$gte': 2006}
                    }]})
        df_b=pd.DataFrame(query)
        lista.append(len(df_b))
        
    return lista
lista = find_founded_year(companies, 2000)

KeyError: 0

In [215]:
                                                        #       MEDIR DISTANCIAS   ---- LATER ON
def distances_c(df,radio_max_meters=10000):  
    lista = []
    for i in range(len(df)):
        a=db.companies_coor.aggregate([
            { "$geoNear": {
                "near": {                    # near: The point for which to find the closest documents.
                    "$location": df[i]
                }, 
                "maxDistance": radio_max_meters,    #The maximum distance from the center point that the documents can be
                "spherical": False,
                "distanceField": "distance" , # The output field that contains the calculated distance.
                "query": { "category_code": "web"},
                "num": 1,
            }}
            ])
        df_b=pd.DataFrame(a)
        lista.append(len(df_b))
    return lista
        
        
distances = list(distances_c(companies,))
print(distances)


KeyError: 0

In [210]:
# https://developers.google.com/places/web-service/search

In [ ]:

# REQUEST TRAFFIC API
    https://traffic.api.here.com/traffic/6.2/flow/xml/8/134/86
    ?app_id={YOUR_APP_ID}
    &app_code={YOUR_APP_CODE}
#https://developer.here.com/documentation/traffic/topics_v6.1/example-flow.html


In [216]:
import googlemaps
import pprint
import time
import requests
import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("G_CLOUD_API_KEY")

In [ ]:
# GOOGLEPLACES --- 
def get_lat_long_places(name):   # name as bus stops
    try:
        query = "{}".format(name)
        print(query)
        # location format :   -33.8670522,151.1957362  tengo que meter latitud, longitud
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type=restaurant&keyword=cruise&key={}".format(latitud,longitud,radius,key)
        response = requests.get(url)
        data = response.json()
        location = data.get('results')[0].get('geometry').get('location')
        return{
            "latitud" : location.get('lat'),
            "longitud" : location.get('lng')
        }
    except: 
        return {
            "latitud" : None,
            "longitud" : None
        }